In [1]:
import pandas as pd
import numpy as np
from scipy.special import softmax
import json

In [2]:
# 2nd round Pseudo-labeling  final best result
convnext_path = "../final_result/convnext_large_pseduo_5/final_logits.csv"
deit_path = "../final_result/deit_large_384_bf_pseduo_5_epoch13/final_logits.csv"
volo_path = "../final_result/volo_d5_512_second_8/final_logits.csv"

In [ ]:
# # 1st round Pseudo-labeling
# convnext_path = "../final_result/convnext_large_pseduo_5/final_logits.csv"
# deit_path = "../final_result/deit_large_384_bf_pseduo_5_epoch13/final_logits.csv"
# volo_path = "../final_result/volo_d5_512/final_logits.csv"

In [ ]:
# # No Pseudo-labeling
# convnext_path = "../final_result/convnext_large_224/final_logits.csv"
# deit_path = "../final_result/deit_large_384_bf/final_logits.csv"
# volo_path = "../final_result/volo_d5_512/final_logits.csv"

In [3]:
convnext_data = pd.read_csv(convnext_path)
deit_data = pd.read_csv(deit_path)    
volo_data = pd.read_csv(volo_path)    

In [4]:
weight1 = 0.35
weight2 = 0.3
weight3 = 0.35

In [5]:
mix_logits = weight3 * np.array(convnext_data.iloc[:, 1:])+weight1* np.array(deit_data.iloc[:, 1:])+weight2* np.array(volo_data.iloc[:, 1:])
pre_label = np.argmax(mix_logits,axis=1)

submit = pd.DataFrame()
# ObservationId,ClassId
submit['imgs'] = convnext_data['imgs']
submit['pred'] = pre_label
class_to_index_reverse = {0:'aeroplane',1:'bicycle',2:'boat',3:'bus',4:'car',5:'chair',6:'diningtable',7:'motorbike',8:'sofa',9:'train'}
submit['pred'] = submit['pred'].map(class_to_index_reverse)

#submit.to_csv('../final_result/best_result/results.csv', index=False, header=True)

In [8]:
# Modify the label by the aspect ratio. Smaller means taller and shorter, more like chair

best_det = "../cascade_rcnn_r50_fpn_1x_coco_backbone_convnextLarge_OnlyAdamW_cos_colorjitter_softmax_corrupt_073_final_cls/cla_final.json"

mix_logits = weight3 * np.array(convnext_data.iloc[:, 1:])+weight1* np.array(deit_data.iloc[:, 1:])+weight2* np.array(volo_data.iloc[:, 1:])
#mix_logits = weight * softmax(np.array(convnext_ood_list[k].iloc[:, 2:]),axis=1)  +(1-weight)* softmax(np.array(deit_ood_list[k].iloc[:, 2:]),axis=1)
pre_label = np.argmax(mix_logits,axis=1)

soft_max_logits = softmax(mix_logits,axis=1)
submit = pd.DataFrame()
# ObservationId,ClassId
submit['imgs'] = convnext_data['imgs']
submit['pred'] = pre_label
group_scores = pd.concat([submit['imgs'], submit['pred'], pd.DataFrame(mix_logits)], axis=1)

# index_to_class = {0:'aeroplane',1:'bicycle',2:'boat',3:'bus',4:'car',5:'chair',6:'diningtable',7:'motorbike',8:'sofa',9:'train'}
        
# pred:bicycle change:chair
bicycle_data = group_scores[group_scores['pred'] == 1]
for item in bicycle_data.iterrows():
    if item[1][5] * 1.8 > item[1][1]:
        submit.loc[item[0],'pred'] = 5

#pred trian; change bus
# baseline: OOD 0.11499 IID 0.3214→0.3212
train_data = group_scores[group_scores['pred'] == 9]
for item in train_data.iterrows():
    if item[1][3] * 1.1 > item[1][9]:
        submit.loc[item[0],'pred'] = 1

#pred: boat; change: bus
boat_data = group_scores[group_scores['pred'] == 2]  
for item in boat_data.iterrows():
        if item[1][3]* 1.5 >item[1][2]:  # 1.5
                #print(item[1]['imgs'])
                submit.loc[item[0],'pred'] = 3
                
#pred: diningtable; change: chair
diningtable_data = group_scores[group_scores['pred'] == 6] 
for item in diningtable_data.iterrows():
        if item[1][5]* 1.3 >item[1][6]: # 1.3
                submit.loc[item[0],'pred']=5

#pred: sofa; change: chair
sofa_data = group_scores[group_scores['pred'] == 8] 
for item in sofa_data.iterrows():
        if item[1][5]* 2.0 >item[1][8]:
                submit.loc[item[0],'pred']= 5
 
#pred: car; change: bus
car_data = group_scores[group_scores['pred'] == 4]
for item in car_data.iterrows():
        if item[1][3]* 1.5 >item[1][4]: 
                submit.loc[item[0],'pred']= 3

# pred:motorbike change:car
motorbike_data = group_scores[group_scores['pred'] == 7]
for item in motorbike_data.iterrows():
        if item[1][4] * 1.3 > item[1][7]:
                submit.loc[item[0],'pred']= 4

# Adjust chair and sofa according to aspect ratio
sofa_data = group_scores[submit['pred'] == 8]
f=open(best_det)
data=json.load(f)
for item in sofa_data.iterrows():
        name_list = item[1]['imgs'].split('.')
        name = name_list[0]
        #print(name)
        #print(item[1]['label'])
        for onedata in data:
                if onedata['image_id'] == name:
                        #print(onedata)
                        if onedata['bbox'][2]/onedata['bbox'][3] < .6 : # setting 0.6
                                submit.loc[item[0],'pred']=5
                                
                        break

        # print(item[1]['label'])
        # print(item[1]['pred'])
all_num = 0
for num in submit['pred'] - pre_label :
        if num == 0:
                all_num+=1
class_to_index_reverse = {0:'aeroplane',1:'bicycle',2:'boat',3:'bus',4:'car',5:'chair',6:'diningtable',7:'motorbike',8:'sofa',9:'train'}
submit['pred'] = submit['pred'].map(class_to_index_reverse)

submit.to_csv('../final_result/best_result/bbox_results.csv', index=False, header=True)
print("change: ",len(submit) - all_num)

change:  543


In [9]:
# Get the best result
submit.to_csv('../final_result/best_result/results.csv', index=False, header=True)

In [13]:
# # Save the image with high confidence as pseudo label
# class_to_index_reverse = {'aeroplane':0,'bicycle':1,'boat':2,'bus':3,'car':4,'chair':5,'diningtable':6,'motorbike':7,'sofa':8,'train':9}

# group_scores = pd.concat([submit['imgs'], submit['pred'], pd.DataFrame(soft_max_logits)], axis=1)

# num_th = 0

# pseudo_label = pd.DataFrame()

# for key,value in group_scores.iterrows(): 
#     confi = value[class_to_index_reverse[value['pred']]]
#     #print(value)
#     #print(value)
#     if confi > 0.8:
#         num_th +=1 
#         pseudo_label = pd.concat([pseudo_label,value],axis=1)
    
    
# num_th

# pseudo_label.T.to_csv('../../data/pseudo/second_confi_8.csv',index =None)

10202